```{contents}
```
## Function Calling

### What Function Calling Is

**Function calling** is a capability where an LLM **returns a structured request to invoke a function** (with validated arguments) instead of producing free-text output. The host application executes the function and feeds the result back to the model to complete the response.

> Function calling turns an LLM into a **controller** that decides *when* to compute or fetch data, while the application performs the action deterministically.

---

### Why Function Calling Exists

LLMs alone are weak at:

* Exact calculations
* Live data access
* Database/API interactions
* Deterministic side effects

Function calling enables:

* **Accuracy** (no guessing)
* **Fresh data** (runtime execution)
* **Safety** (schema-validated inputs)
* **Auditability** (logged calls)

---

### High-Level Flow

```
User Input
   ↓
LLM decides a function is needed
   ↓
Structured function call (JSON)
   ↓
Application executes function
   ↓
Function result returned to LLM
   ↓
LLM produces final answer
```

---

### Core Concepts

#### Function (Tool)

A deterministic routine (API, DB query, calculator).

#### Function Schema

A JSON schema describing:

* Name
* Description
* Parameters and types

#### Function Call

A model-generated JSON object selecting a function and arguments.

#### Observation

The function’s output passed back to the model.

---

### Function Calling vs Prompt-Only “Tools”

| Aspect               | Prompt-Only | Function Calling |
| -------------------- | ----------- | ---------------- |
| Structure            | Free text   | Strict JSON      |
| Validation           | ❌           | ✅                |
| Determinism          | ❌           | ✅                |
| Safety               | Low         | High             |
| Production readiness | Low         | High             |

---

### Minimal Demonstration (Conceptual)

#### 1) Define a Function

```python
def ticket_count(source: str) -> int:
    return 128
```

#### 2) Describe the Function (Schema)

```json
{
  "name": "ticket_count",
  "description": "Return total number of tickets for a given source",
  "parameters": {
    "type": "object",
    "properties": {
      "source": { "type": "string" }
    },
    "required": ["source"]
  }
}
```

#### 3) Model Emits a Function Call

```json
{
  "name": "ticket_count",
  "arguments": { "source": "jira" }
}
```

#### 4) Application Executes and Returns Result

```json
{ "result": 128 }
```

#### 5) Model Finalizes the Answer

```
There are 128 tickets in Jira.
```

---

### Function Calling in LangChain

LangChain standardizes function calling across providers.

### Define a Tool (Function)

```python
from langchain.tools import tool

@tool
def ticket_count(source: str) -> int:
    """Return total number of tickets for a given source."""
    return 128
```

LangChain auto-derives:

* Name
* Description
* Input schema

---

### Bind Functions to a Chat Model

```python
llm_with_tools = llm.bind_tools([ticket_count])
```

Now the model can choose to call `ticket_count`.

---

### Invoke

```python
response = llm_with_tools.invoke(
    "How many tickets are there in Jira?"
)
```

LangChain:

* Parses the function call
* Executes it
* Feeds the result back
* Returns the final answer

---

### Function Calling vs Tool Calling vs Agents

| Concept          | Purpose              | Steps |
| ---------------- | -------------------- | ----- |
| Function Calling | Invoke a function    | 1–2   |
| Tool Calling     | Abstracted functions | 1–2   |
| Agent (ReAct)    | Multi-step reasoning | Many  |

* **Function calling** is the primitive.
* **Tool calling** is a framework abstraction.
* **Agents** orchestrate many calls.

---

### Common Use Cases

* Database reads
* Calculations
* Fetching live metrics
* Validation checks
* Simple workflows

---

### Guardrails & Best Practices

* Use **strict schemas**
* Keep functions **single-purpose**
* Separate **read** vs **write** functions
* Log all calls
* Set **low temperature**
* Add approvals for destructive actions

---

### Failure Modes (and Fixes)

* **Wrong function chosen** → improve descriptions
* **Bad arguments** → tighter schemas
* **Over-calling** → system prompt constraints
* **Loops** → max call limits

---

### Interview-Ready Summary

> “Function calling allows an LLM to emit structured, schema-validated requests to invoke external functions. It enables deterministic actions, accurate computations, and safe integrations, and serves as the foundation for tool calling and agentic workflows.”

---

### Rule of Thumb

* **Exact data/action needed → Function calling**
* **Multiple actions → Agents**
* **Complex workflows → State machines (e.g., LangGraph)**
